# Combining QuPath and Deepcell segmentation results

# Prerequisites

In [2]:
library(imcRtools)
library(tidyverse)

Loading required package: SpatialExperiment

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

In [3]:
set_base_dir_method <- "Relative"

In [4]:
if (set_base_dir_method == "Relative") {
    notebook_dir <- getwd()
    data_dir <- fs::path_dir(notebook_dir)
} else if (set_base_dir_method == "Manual") {
    data_dir = "/path/to/data/directory"
}

In [5]:
# set the base directory
base_dir = data_dir

In [6]:
results_dir = fs::path_join(c(base_dir, "analysis"))
results_graphs_dir = fs::path_join(c(base_dir, "analysis", "graphs"))
results_objects_dir = fs::path_join(c(base_dir, "analysis", "objects"))
cell_table_dir_qupath = fs::path_join(c(base_dir, "segmentation","cell_table_qupath"))
cell_table_dir_qc = fs::path_join(c(base_dir, "segmentation", "cell_table"))

In [7]:
for (directory in c(results_dir, results_graphs_dir, results_objects_dir)) {
    dir.create(directory,showWarnings = FALSE)
}

In [8]:
spe_Deepcell_old <- readRDS(fs::path_join(c(results_objects_dir,"spe_DeepCell_old.rds")))
spe_QuPath_ananya <- readRDS(fs::path_join(c(results_objects_dir,"spe_QuPath_ananya_QuPath.rds")))


In [9]:
rownames(assays(spe_QuPath_ananya)[["exprs"]])[!(rownames(assays(spe_QuPath_ananya)[["exprs"]]) %in% rownames(assays(spe_Deepcell_old)[["exprs"]]))]

rownames(assays(spe_Deepcell_old)[["exprs"]])[!(rownames(assays(spe_Deepcell_old
                                                                )[["exprs"]]) %in% rownames(assays(spe_QuPath_ananya)[["exprs"]]))]



character(0)

character(0)

In [10]:
data_Deepcell_ananya <- as.data.frame(colData(spe_Deepcell_old))
selected_data <- cbind(data_Deepcell_ananya, spatialCoords(spe_Deepcell_old))
cur_exprs <- assay(spe_Deepcell_old, "exprs")
cur_exprs <- as.data.frame(t(cur_exprs))
data_Deepcell_ananya <- cbind(selected_data, cur_exprs)


data_QuPath_ananya <- as.data.frame(colData(spe_QuPath_ananya))
selected_data <- cbind(data_QuPath_ananya, spatialCoords(spe_QuPath_ananya))
cur_exprs <- assay(spe_QuPath_ananya, "exprs")
cur_exprs <- as.data.frame(t(cur_exprs))
data_QuPath_ananya <- cbind(selected_data, cur_exprs)

data_Deepcell_ananya$ObjectNumber <- as.character(data_Deepcell_ananya$ObjectNumber)


combined_data <- bind_rows(data_Deepcell_ananya,data_QuPath_ananya, .id = "source") 


                                   
combined_data$source <- recode(combined_data$source ,"1"="Deepcell", "2"="QuPath")

In [11]:
saveRDS(combined_data,fs::path_join(c(results_objects_dir,"combined_data.rds")))
write.csv(combined_data, fs::path_join(c(results_dir, "combined_data_export.csv")))